# Script d'insertion des données dans la table "games_publishers"

In [ ]:
import ast
import pandas as pd
import mysql.connector

In [ ]:
from config import config

In [ ]:
descriptions_path = "../data/games_description.csv"

df_descriptions = pd.read_csv(descriptions_path)

In [ ]:
df_descriptions

### Récupération des données dans la base

In [ ]:
# Connexion à la base

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [ ]:
# Récupération de la map game > id

cursor.execute("SELECT id, name FROM games")
game_id_map = {name: id_ for id_, name in cursor.fetchall()}

In [ ]:
# Récupération de la map publisher > id

cursor.execute("SELECT id, publisher FROM publishers")
publisher_id_map = {publisher: id_ for id_, publisher in cursor.fetchall()}

### Préparation des données pour insertion

In [ ]:
# Conversion des strings en listes pour les éditeurs

df_descriptions['publisher'] = df_descriptions['publisher'].apply(ast.literal_eval)

In [ ]:
# On crée les paires games/publishers

game_publisher_pairs = []
for _, row in df_descriptions.iterrows():
    game_name = row['name']
    game_id = game_id_map.get(game_name)
    if game_id is not None:
        for publisher in row['publisher']:
            publisher_id = publisher_id_map.get(publisher)
            if publisher_id is not None:
                game_publisher_pairs.append((game_id, publisher_id))

### Insertion des données

In [ ]:
# Requête SQL

insert_query = "INSERT IGNORE INTO games_publishers (game_id, publisher_id) VALUES (%s, %s)"

In [ ]:
# Exécution de la requête et fin de connexion

cursor.executemany(insert_query, game_publisher_pairs)

cnx.commit()
cursor.close()
cnx.close()